In [61]:
import pandas as pd
import matplotlib.pyplot as plt
# Create a discussion file path list
discussion_file1_path = "../Week1/DevGPT/snapshot_20230727/20230727_195954_discussion_sharings.json"
discussion_file2_path = "../Week1/DevGPT/snapshot_20230803/20230803_094811_discussion_sharings.json"
discussion_file3_path = "../Week1/DevGPT/snapshot_20230810/20230810_124048_discussion_sharings.json"
discussion_file4_path = "../Week1/DevGPT/snapshot_20230817/20230817_130721_discussion_sharings.json"
discussion_file5_path = "../Week1/DevGPT/snapshot_20230824/20230824_102000_discussion_sharings.json"
discussion_file6_path = "../Week1/DevGPT/snapshot_20230831/20230831_061926_discussion_sharings.json"

discussion_json_files = [discussion_file1_path,discussion_file2_path,discussion_file3_path,discussion_file4_path,discussion_file5_path,discussion_file6_path]

In [62]:
# Create an issue file path list
issue_file1_path = "../Week1/DevGPT/snapshot_20230727/20230727_195941_issue_sharings.json"
issue_file2_path = "../Week1/DevGPT/snapshot_20230803/20230803_094705_issue_sharings.json"
issue_file3_path = "../Week1/DevGPT/snapshot_20230810/20230810_123938_issue_sharings.json"
issue_file4_path = "../Week1/DevGPT/snapshot_20230817/20230817_130502_issue_sharings.json"
issue_file5_path = "../Week1/DevGPT/snapshot_20230824/20230824_101836_issue_sharings.json"
issue_file6_path = "../Week1/DevGPT/snapshot_20230831/20230831_061759_issue_sharings.json"

issue_json_files = [issue_file1_path,issue_file2_path,issue_file3_path,issue_file4_path,issue_file5_path,issue_file6_path]

In [63]:
# Concatenate two types of files
json_files = discussion_json_files + issue_json_files
json_files

['../Week1/DevGPT/snapshot_20230727/20230727_195954_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230803/20230803_094811_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230810/20230810_124048_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230817/20230817_130721_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230824/20230824_102000_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230831/20230831_061926_discussion_sharings.json',
 '../Week1/DevGPT/snapshot_20230727/20230727_195941_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230803/20230803_094705_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230810/20230810_123938_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230817/20230817_130502_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230824/20230824_101836_issue_sharings.json',
 '../Week1/DevGPT/snapshot_20230831/20230831_061759_issue_sharings.json']

In [64]:
# Load the data into DataFrame from all files 
dataframes = []
for file in json_files:
    try:
        # load json files
        df = pd.read_json(file)
        dataframes.append(df["Sources"])
    except ValueError as e:
        print(f"error from {file} : {e}")

# combine all DataFrames
combined_df = pd.concat(dataframes, ignore_index=True)
print(combined_df.info())
print(combined_df.head())

<class 'pandas.core.series.Series'>
RangeIndex: 1998 entries, 0 to 1997
Series name: Sources
Non-Null Count  Dtype 
--------------  ----- 
1998 non-null   object
dtypes: object(1)
memory usage: 15.7+ KB
None
0    {'Type': 'discussion', 'URL': 'https://github....
1    {'Type': 'discussion', 'URL': 'https://github....
2    {'Type': 'discussion', 'URL': 'https://github....
3    {'Type': 'discussion', 'URL': 'https://github....
4    {'Type': 'discussion', 'URL': 'https://github....
Name: Sources, dtype: object


In [65]:
import pandas as pd

# Initialize lists to collect extracted data
Type_list = []
RepoLanguage_list = []
Number_list = []
UpvoteCount_list = []
Conversations_num_list = []
PromptTokens_list = []
AnswerTokens_list = []
Newline=[]

# Iterate through the rows of the DataFrame
for i in range(len(combined_df)):
    # Extract Type, RepoLanguage, Number, and UpvoteCount from the current row
    row = combined_df.iloc[i]
    type_value = row.get("Type", "unknown")  # Extract "Type" field
    repo_language = row.get("RepoLanguage", "unknown")
    number = row.get("Number", 0)
    upvote_count = row.get("UpvoteCount", 0)
    
    # Extract ChatgptSharing field
    ChatgptSharing = row.get("ChatgptSharing", [])
    total_conversations = 0  # Counter for the number of conversations in a record
    
    # Process each ChatgptSharing item
    for item in ChatgptSharing:
        Conversations = item.get("Conversations", [])
        total_conversations += len(Conversations)  # Count the number of conversation turns
        
        # Process each conversation (Prompt and Answer)
        for conv in Conversations:
            prompt = conv.get("Prompt", "")
            answer = conv.get("Answer", "")
            
            # Count tokens using split()
            prompt_tokens = len(prompt.split())  # Splitting on spaces
            answer_tokens = len(answer.split())

            prompt_newline_count = prompt.count("\n\n")
            # Append **per conversation** (ensuring matching lengths)
            Type_list.append(type_value)  # Append type for each conversation
            RepoLanguage_list.append(repo_language)
            Number_list.append(number)
            UpvoteCount_list.append(upvote_count)
            Conversations_num_list.append(total_conversations)  # Append total conv count per conversation
            PromptTokens_list.append(prompt_tokens)
            AnswerTokens_list.append(answer_tokens)
            Newline.append( prompt_newline_count)

# Combine the extracted data into a DataFrame
extracted_data = {
    "Type": Type_list,
    "RepoLanguage": RepoLanguage_list,
    "Number": Number_list,
    "UpvoteCount": UpvoteCount_list,
    "PromptTokens": PromptTokens_list,
    "AnswerTokens": AnswerTokens_list,
    "Newline": Newline
}
df = pd.DataFrame(extracted_data)
# Display the resulting DataFrame
df.isnull().sum()
df["RepoLanguage"]=df["RepoLanguage"].fillna("other")
print(df.isnull().sum())
df["Type"].unique()


Type            0
RepoLanguage    0
Number          0
UpvoteCount     0
PromptTokens    0
AnswerTokens    0
Newline         0
dtype: int64


array(['discussion', 'issue'], dtype=object)

In [66]:
import pandas as pd
import re

# Initialize lists to collect extracted data
Type_list = []
RepoLanguage_list = []
Number_list = []
UpvoteCount_list = []
Conversations_num_list = []
PromptTokens_list = []
AnswerTokens_list = []
Newline = []
KeywordCount_list = []  # New list to store single-word keyword count

# Define the list of single-word keywords
keywords = [
    # Instructional & Step-by-Step Words
    "list", "enumerate", "examples", "overview", "summary", "keypoints", "factors",
    "explain", "describe", "steps", "process", "how", "why", "detailed", "tutorial",
    "instruction", "methodology", "workflow", "structure",

    # Comparative & Analytical Terms
    "comparison", "contrast", "difference", "analysis", "evaluate", "impact", 
    "relationship", "correlation", "variance", "distribution", "hypothesis",
    "assessment", "benchmark", "criteria", "metrics", "performance",

    # Theoretical & Conceptual Words
    "reasoning", "theory", "principles", "fundamentals", "concepts", "philosophy",
    "mechanism", "frameworks", "paradigm", "approach", "perspective",

    # Machine Learning & AI Terms
    "gradient", "deep", "learning", "neural", "networks", "optimization", 
    "regularization", "probability", "statistics", "machine", "transformers",
    "classification", "prediction", "reinforcement", "hyperparameters",
    "activation", "backpropagation", "clustering", "feature", "parameter",
    "regression", "outliers", "sampling", "estimation", "loss function",
    "interpretability", "generalization", "convergence",

    # Example & Scenario-Based Words
    "example", "use case", "scenario", "application", "illustration", 
    "simulation", "demonstration", "real-world", "experiment",

    # Advanced Technical Terms
    "schedule", "implementation", "explanation", "hypothesis", "derivatives",
    "decision tree", "random forest", "support vector", "Bayesian", "Markov",
    "algorithm", "model", "training", "evaluation", "reinforcement learning"
]


# Compile regex pattern for single-word keyword matching
pattern = r'\b(?:' + '|'.join(map(re.escape, keywords)) + r')\b'

# Iterate through the DataFrame rows
for i in range(len(combined_df)):
    # Extract row values for Type, RepoLanguage, Number, and UpvoteCount
    row = combined_df.iloc[i]
    type_value = row.get("Type", "unknown")  
    repo_language = row.get("RepoLanguage", "unknown")
    number = row.get("Number", 0)
    upvote_count = row.get("UpvoteCount", 0)
    
    # Extract ChatgptSharing field
    ChatgptSharing = row.get("ChatgptSharing", [])
    total_conversations = 0  # Counter for conversation count per record
    
    # Process each ChatgptSharing item
    for item in ChatgptSharing:
        Conversations = item.get("Conversations", [])
        total_conversations += len(Conversations)  # Count number of conversations
        
        # Process each conversation (Prompt and Answer)
        for conv in Conversations:
            prompt = conv.get("Prompt", "")
            answer = conv.get("Answer", "")

            # Count tokens in Prompt and Answer
            prompt_tokens = len(prompt.split())  
            answer_tokens = len(answer.split())

            # Count occurrences of newline ("\n\n") in the prompt
            prompt_newline_count = prompt.count("\n\n")

            # Count the number of single-word keywords in the Prompt
            prompt_keyword_count = len(re.findall(pattern, prompt.lower())) if prompt else 0

            # Append values for each conversation
            Type_list.append(type_value)  
            RepoLanguage_list.append(repo_language)
            Number_list.append(number)
            UpvoteCount_list.append(upvote_count)
            Conversations_num_list.append(total_conversations)  
            PromptTokens_list.append(prompt_tokens)
            AnswerTokens_list.append(answer_tokens)
            Newline.append(prompt_newline_count)
            KeywordCount_list.append(prompt_keyword_count)  

# Combine extracted data into a DataFrame
extracted_data = {
    "Type": Type_list,
    "RepoLanguage": RepoLanguage_list,
    "Number": Number_list,
    "UpvoteCount": UpvoteCount_list,
    "PromptTokens": PromptTokens_list,
    "AnswerTokens": AnswerTokens_list,
    "Newline": Newline,
    "KeywordCount": KeywordCount_list  # New column for keyword count
}

df = pd.DataFrame(extracted_data)

# Fill missing values in RepoLanguage
df["RepoLanguage"] = df["RepoLanguage"].fillna("other")

# Display summary of missing values and unique Type values
print(df.isnull().sum())
print(df.head())
print(df["Type"].unique())


Type            0
RepoLanguage    0
Number          0
UpvoteCount     0
PromptTokens    0
AnswerTokens    0
Newline         0
KeywordCount    0
dtype: int64
         Type RepoLanguage  Number  UpvoteCount  PromptTokens  AnswerTokens  \
0  discussion        other      27            1            15           341   
1  discussion        other      27            1            15           341   
2  discussion        other      74            1          1690           184   
3  discussion        other      74            1           369            58   
4  discussion       Python       7            1           358            11   

   Newline  KeywordCount  
0        0             0  
1        0             0  
2       84            53  
3       17             6  
4       11             2  
['discussion' 'issue']


In [67]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
label_encoders = {}
for col in ["Type", "RepoLanguage"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le 
print(df.head())

   Type  RepoLanguage  Number  UpvoteCount  PromptTokens  AnswerTokens  \
0     0            34      27            1            15           341   
1     0            34      27            1            15           341   
2     0            34      74            1          1690           184   
3     0            34      74            1           369            58   
4     0            23       7            1           358            11   

   Newline  KeywordCount  
0        0             0  
1        0             0  
2       84            53  
3       17             6  
4       11             2  


In [72]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Define predictors (X) and target variable (y)
X = df.drop(columns=["AnswerTokens"])
y = df["AnswerTokens"]  

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=65)

param_grid = {"n_neighbors": np.arange(1, 30, 1)}  # Try k values from 1 to 29 (odd numbers)

knn = KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring="neg_mean_squared_error")
grid_search.fit(X_train, y_train)

# Get the best k value
best_k = grid_search.best_params_["n_neighbors"]
print(f"Best k value found: {best_k}")

knn_best = KNeighborsRegressor(n_neighbors=best_k)
knn_best.fit(X_train, y_train)

y_pred = knn_best.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)


y_test_no_zeros = y_test.replace(0, 1)
mape = (abs((y_test - y_pred) / y_test_no_zeros).mean()) * 100  
accuracy = 100 - mape  # Accuracy derived from MAPE

print(f"Root Test Mean Squared Error (MSE): {rmse:.4f}")
print(f"Test R-squared (R²): {r2:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"Prediction Accuracy: {accuracy:.2f}%")

Best k value found: 1
Root Test Mean Squared Error (MSE): 34.1366
Test R-squared (R²): 0.9157
Mean Absolute Percentage Error (MAPE): 15.82%
Prediction Accuracy: 84.18%
